# California Housing Dataset - Modélisation

Dans ce notebook, nous allons :
1. Implémenter le pipeline de preprocessing
2. Comparer différents modèles
3. Optimiser les hyperparamètres
4. Évaluer le modèle final

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

from california_housing.preprocessing.preprocessor import HousingPreprocessor, prepare_data
from california_housing.utils.evaluation import evaluate_model, format_metrics

%matplotlib inline
plt.style.use('seaborn')
sns.set_palette('husl')

## 1. Préparation des Données

In [ ]:
# Chargement et split des données
data = fetch_california_housing(as_frame=True)
df = data.frame.copy()

X, y = prepare_data(df)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Dimensions des données:")
print(f"X_train: {X_train.shape}")
print(f"X_test: {X_test.shape}")

## 2. Preprocessing

In [ ]:
# Configuration et application du preprocessor
preprocessor = HousingPreprocessor(handle_outliers=True, handle_skewness=True)
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

if preprocessor.skewed_features:
    print("Features avec forte asymétrie traitées:")
    print(preprocessor.skewed_features)

## 3. Comparaison des Modèles

In [ ]:
# Définition des modèles à comparer
models = {
    'Linear Regression': LinearRegression(),
    'Ridge': Ridge(),
    'Random Forest': RandomForestRegressor(random_state=42),
    'SVR': SVR(),
    'KNN': KNeighborsRegressor()
}

# Évaluation initiale
results = []
for name, model in models.items():
    print(f"\nÉvaluation de {name}...")
    model.fit(X_train_processed, y_train)
    metrics = evaluate_model(model, X_test_processed, y_test)
    results.append({
        'Model': name,
        **format_metrics(metrics)
    })

results_df = pd.DataFrame(results).set_index('Model')
display(results_df)

## 4. Optimisation des Hyperparamètres

In [ ]:
# Grilles de paramètres pour GridSearchCV
param_grids = {
    'Ridge': {
        'alpha': [0.1, 1.0, 10.0],
        'solver': ['auto', 'svd', 'cholesky']
    },
    'Random Forest': {
        'n_estimators': [100, 200],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5]
    },
    'SVR': {
        'C': [0.1, 1.0, 10.0],
        'kernel': ['rbf', 'linear'],
        'epsilon': [0.1, 0.2]
    },
    'KNN': {
        'n_neighbors': [3, 5, 7],
        'weights': ['uniform', 'distance']
    }
}

# Optimisation des meilleurs modèles
best_models = {}
tuning_results = []

for name, model in models.items():
    if name == 'Linear Regression':
        continue
        
    print(f"\nOptimisation de {name}...")
    grid = GridSearchCV(
        model,
        param_grids[name],
        cv=5,
        scoring='r2',
        n_jobs=-1
    )
    grid.fit(X_train_processed, y_train)
    
    best_models[name] = grid.best_estimator_
    metrics = evaluate_model(grid.best_estimator_, X_test_processed, y_test)
    
    tuning_results.append({
        'Model': name,
        'Best Parameters': grid.best_params_,
        **format_metrics(metrics)
    })

tuning_df = pd.DataFrame(tuning_results).set_index('Model')
display(tuning_df)

## 5. Analyse du Meilleur Modèle

In [ ]:
# Sélection du meilleur modèle
best_model_name = tuning_df['R²'].astype(str).str.extract('(\d+\.\d+)').astype(float).idxmax()
best_model = best_models[best_model_name]

print(f"Meilleur modèle : {best_model_name}")

# Visualisation des prédictions
y_pred = best_model.predict(X_test_processed)

plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Valeurs Réelles')
plt.ylabel('Prédictions')
plt.title(f'Prédictions vs Réalité - {best_model_name}')
plt.show()

# Analyse des résidus
residuals = y_test - y_pred

plt.figure(figsize=(10, 6))
plt.scatter(y_pred, residuals, alpha=0.5)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Prédictions')
plt.ylabel('Résidus')
plt.title('Analyse des Résidus')
plt.show()

## 6. Importance des Features

In [ ]:
if hasattr(best_model, 'feature_importances_'):
    importances = pd.Series(
        best_model.feature_importances_,
        index=X.columns
    ).sort_values(ascending=True)
    
    plt.figure(figsize=(10, 6))
    importances.plot(kind='barh')
    plt.title('Importance des Features')
    plt.show()

## 7. Conclusions

De cette analyse de modélisation, nous pouvons conclure :

1. Performance des modèles :
   - Comparaison des différents algorithmes
   - Impact de l'optimisation des hyperparamètres

2. Meilleur modèle :
   - Algorithme sélectionné
   - Hyperparamètres optimaux
   - Métriques de performance

3. Features importantes :
   - Variables les plus prédictives
   - Potentielles pistes d'amélioration